In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.optim import lr_scheduler
import torch.optim as optim
from torchvision import datasets, transforms

import math, os
import matplotlib.pyplot as plt



## Network Definitions

In [9]:
'''
Overarching Model
'''
class CapsNet(nn.Module):
    def __init__(self, routing_iterations, n_classes=10):
        super(CapsNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 256, kernel_size=9, stride=1)
        
        self.primaryCaps = PrimaryCapsLayer(256, 32, 8, kernel_size=9, stride=2)  # outputs: normalized sheets
        
        self.num_primaryCaps = 32 * 6 * 6
        routing_module = AgreementRouting(self.num_primaryCaps, n_classes, routing_iterations)
        self.digitCaps = CapsLayer(self.num_primaryCaps, 8, n_classes, 16, routing_module)

    def forward(self, input):
        x = self.conv1(input)
        x = F.relu(x)
        x = self.primaryCaps(x) # output sheet
        x = self.digitCaps(x) # output score matrix (before final flat)
        probs = x.pow(2).sum(dim=2).sqrt()
        return x, probs 

In [10]:
'''
Input: FM after 1st convolution
Output: Normalized sheet
'''
class PrimaryCapsLayer(nn.Module):
    def __init__(self, input_channels, output_caps, output_dim, kernel_size, stride):
        super(PrimaryCapsLayer, self).__init__()
        self.conv = nn.Conv2d(input_channels, output_caps * output_dim, kernel_size=kernel_size, stride=stride)
        self.input_channels = input_channels
        self.output_caps = output_caps
        self.output_dim = output_dim

    def forward(self, input):
        out = self.conv(input)
        N, C, H, W = out.size()
        out = out.view(N, self.output_caps, self.output_dim, H, W) # splitting into capsules

        # will output N x OUT_CAPS x OUT_DIM
        out = out.permute(0, 1, 3, 4, 2).contiguous() 
        out = out.view(out.size(0), -1, out.size(4)) # flat N*sheet
        out = squash(out) # normalize
        return out

In [11]:
def squash(x):
    lengths2 = x.pow(2).sum(dim=2)
    lengths = lengths2.sqrt()
    x = x * (lengths2 / (1 + lengths2) / lengths).view(x.size(0), x.size(1), 1)
    return x


class AgreementRouting(nn.Module):
    def __init__(self, input_caps, output_caps, n_iterations):
        super(AgreementRouting, self).__init__()
        self.n_iterations = n_iterations
        self.b = nn.Parameter(torch.zeros((input_caps, output_caps)))

    def forward(self, u_predict):
        batch_size, input_caps, output_caps, output_dim = u_predict.size()
        
        c = F.softmax(self.b)
        s = (c.unsqueeze(2) * u_predict).sum(dim=1)
        v = squash(s)

        if self.n_iterations > 0:
            b_batch = self.b.expand((batch_size, input_caps, output_caps))
            for r in range(self.n_iterations):
                v = v.unsqueeze(1)
                b_batch = b_batch + (u_predict * v).sum(-1)

                c = F.softmax(b_batch.view(-1, output_caps)).view(-1, input_caps, output_caps, 1)
                s = (c * u_predict).sum(dim=1)
                v = squash(s)

        return v


class CapsLayer(nn.Module):
    def __init__(self, input_caps, input_dim, output_caps, output_dim, routing_module):
        super(CapsLayer, self).__init__()
        self.input_dim = input_dim
        self.input_caps = input_caps
        self.output_dim = output_dim
        self.output_caps = output_caps
        self.weights = nn.Parameter(torch.Tensor(input_caps, input_dim, output_caps * output_dim))
        self.routing_module = routing_module
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.input_caps)
        self.weights.data.uniform_(-stdv, stdv)

    def forward(self, caps_output):
        caps_output = caps_output.unsqueeze(2)
        u_predict = caps_output.matmul(self.weights)
        u_predict = u_predict.view(u_predict.size(0), self.input_caps, self.output_caps, self.output_dim)
        v = self.routing_module(u_predict)
        return v


In [12]:
### Reconstruction Stuff 
class ReconstructionNet(nn.Module):
    def __init__(self, n_dim=16, n_classes=10):
        super(ReconstructionNet, self).__init__()
        self.fc1 = nn.Linear(n_dim * n_classes, 512)
        self.fc2 = nn.Linear(512, 1024)
        self.fc3 = nn.Linear(1024, 784)
        self.n_dim = n_dim
        self.n_classes = n_classes

    def forward(self, x, target):
        mask = Variable(torch.zeros((x.size()[0], self.n_classes)), requires_grad=False)
        if next(self.parameters()).is_cuda:
            mask = mask.cuda()
        mask.scatter_(1, target.view(-1, 1), 1.)
        mask = mask.unsqueeze(2)
        x = x * mask
        x = x.view(-1, self.n_dim * self.n_classes)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x


class CapsNetWithReconstruction(nn.Module):
    def __init__(self, capsnet, reconstruction_net):
        super(CapsNetWithReconstruction, self).__init__()
        self.capsnet = capsnet
        self.reconstruction_net = reconstruction_net

    def forward(self, x, target):
        x, probs = self.capsnet(x)
        reconstruction = self.reconstruction_net(x, target)
        return reconstruction, probs

## Train

### Helper functions

In [17]:
### Define Loss Functions
class MarginLoss(nn.Module):
    def __init__(self, m_pos, m_neg, lambda_):
        super(MarginLoss, self).__init__()
        self.m_pos = m_pos
        self.m_neg = m_neg
        self.lambda_ = lambda_

    def forward(self, lengths, targets, size_average=True):
        t = torch.zeros(lengths.size()).long()
        if targets.is_cuda:
            t = t.cuda()
        t = t.scatter_(1, targets.data.view(-1, 1), 1)
        targets = Variable(t)
        losses = targets.float() * F.relu(self.m_pos - lengths).pow(2) + \
                 self.lambda_ * (1. - targets.float()) * F.relu(lengths - self.m_neg).pow(2)
        return losses.mean() if size_average else losses.sum()

### save model
def saveModel(model,name,state_dict=False):
    if state_dict:
        torch.save(model.state_dict(),name)
    else:
        torch.save(model,name)
        
    

In [19]:
def train(Epoch,Loss,Rec_loss_every=1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if CUDA:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target, requires_grad=False)
        optimizer.zero_grad()
        if RECONSTRUCTION:
            output, probs = model(data, target)
            reconstruction_loss = F.mse_loss(output, data.view(-1, 784))
            margin_loss = loss_fn(probs, target)
            loss = reconstruction_alpha * reconstruction_loss + margin_loss
        else:
            output, probs = model(data)
            loss = loss_fn(probs, target)
        
        if batch_idx % Rec_loss_every == 0:
            Loss.append(loss.data[0])
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INT == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                Epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if CUDA:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)

        if RECONSTRUCTION:
            output, probs = model(data, target)
            reconstruction_loss = F.mse_loss(output, data.view(-1, 784), size_average=False).data[0]
            test_loss += loss_fn(probs, target, size_average=False).data[0]
            test_loss += reconstruction_alpha * reconstruction_loss
        else:
            output, probs = model(data)
            test_loss += loss_fn(probs, target, size_average=False).data[0]

        pred = probs.data.max(1, keepdim=True)[1]  # get the index of the max probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss

### Defining paramters, loaders, SGD elements

In [26]:
# # Training settings
# parser = argparse.ArgumentParser(description='CapsNet with MNIST')
# parser.add_argument('--batch-size', type=int, default=128, metavar='N',
#                     help='input batch size for training (default: 64)')
# parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
#                     help='input batch size for testing (default: 1000)')
# parser.add_argument('--epochs', type=int, default=250, metavar='N',
#                     help='number of epochs to train (default: 10)')
# parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
#                     help='learning rate (default: 0.01)')
# parser.add_argument('--no-cuda', action='store_true', default=False,
#                     help='disables CUDA training')
# parser.add_argument('--seed', type=int, default=1, metavar='S',
#                     help='random seed (default: 1)')
# parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                     help='how many batches to wait before logging training status')
# parser.add_argument('--routing_iterations', type=int, default=3)
# parser.add_argument('--with_reconstruction', action='store_true', default=False)
# args = parser.parse_args()
# args.cuda = not args.no_cuda and torch.cuda.is_available()


BATCH_SIZE = 200
TEST_BATCH_SIZE = 1000
EPOCHS = 20
LEARNING_RATE = 0.001
CUDA = torch.cuda.is_available()
SEED = 1
LOG_INT = 10 # how many batches to wait before logging training status
ROUTING_ITERS = 3
RECONSTRUCTION = False

kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.Pad(2), transforms.RandomCrop(28),
                       transforms.ToTensor()
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor()
    ])),
    batch_size=TEST_BATCH_SIZE, shuffle=False)

model = CapsNet(ROUTING_ITERS)

if RECONSTRUCTION:
    reconstruction_model = ReconstructionNet(16, 10)
    reconstruction_alpha = 0.0005
    model = CapsNetWithReconstruction(model, reconstruction_model)

if CUDA:
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=5, min_lr=1e-6)
loss_fn = MarginLoss(0.9, 0.1, 0.5)

In [27]:
### Call training functions 
Loss = [], L_every = 1
for epoch in range(1, EPOCHS + 1):
    train(epoch,Loss,L_every)
    test_loss = test()
    scheduler.step(test_loss)
    


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.080883
Train Epoch: 1 [2000/60000 (3%)]	Loss: 0.056401
Train Epoch: 1 [4000/60000 (7%)]	Loss: 0.037683
Train Epoch: 1 [6000/60000 (10%)]	Loss: 0.033561
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.022048
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.015681
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.014932
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.012336
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.009352
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.008090
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.009220
Train Epoch: 1 [22000/60000 (37%)]	Loss: 0.007391
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.005422
Train Epoch: 1 [26000/60000 (43%)]	Loss: 0.004481
Train Epoch: 1 [28000/60000 (47%)]	Loss: 0.007688
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.005959
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.004556
Train Epoch: 1 [34000/60000 (57%)]	Loss: 0.004974
Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.004991
Train Epoch: 1 [38000/60000 (63%)]	Loss: 0.005923
Train Epoch

Train Epoch: 6 [20000/60000 (33%)]	Loss: 0.001204
Train Epoch: 6 [22000/60000 (37%)]	Loss: 0.001131
Train Epoch: 6 [24000/60000 (40%)]	Loss: 0.001336
Train Epoch: 6 [26000/60000 (43%)]	Loss: 0.001258
Train Epoch: 6 [28000/60000 (47%)]	Loss: 0.000944
Train Epoch: 6 [30000/60000 (50%)]	Loss: 0.001601
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.000871
Train Epoch: 6 [34000/60000 (57%)]	Loss: 0.000581
Train Epoch: 6 [36000/60000 (60%)]	Loss: 0.001567
Train Epoch: 6 [38000/60000 (63%)]	Loss: 0.001672
Train Epoch: 6 [40000/60000 (67%)]	Loss: 0.000648
Train Epoch: 6 [42000/60000 (70%)]	Loss: 0.000897
Train Epoch: 6 [44000/60000 (73%)]	Loss: 0.001541
Train Epoch: 6 [46000/60000 (77%)]	Loss: 0.000899
Train Epoch: 6 [48000/60000 (80%)]	Loss: 0.001153
Train Epoch: 6 [50000/60000 (83%)]	Loss: 0.001129
Train Epoch: 6 [52000/60000 (87%)]	Loss: 0.000532
Train Epoch: 6 [54000/60000 (90%)]	Loss: 0.000738
Train Epoch: 6 [56000/60000 (93%)]	Loss: 0.000852
Train Epoch: 6 [58000/60000 (97%)]	Loss: 0.001289


Train Epoch: 11 [36000/60000 (60%)]	Loss: 0.001183
Train Epoch: 11 [38000/60000 (63%)]	Loss: 0.000534
Train Epoch: 11 [40000/60000 (67%)]	Loss: 0.000959
Train Epoch: 11 [42000/60000 (70%)]	Loss: 0.000295
Train Epoch: 11 [44000/60000 (73%)]	Loss: 0.000448
Train Epoch: 11 [46000/60000 (77%)]	Loss: 0.000817
Train Epoch: 11 [48000/60000 (80%)]	Loss: 0.000437
Train Epoch: 11 [50000/60000 (83%)]	Loss: 0.000803
Train Epoch: 11 [52000/60000 (87%)]	Loss: 0.000652
Train Epoch: 11 [54000/60000 (90%)]	Loss: 0.000372
Train Epoch: 11 [56000/60000 (93%)]	Loss: 0.000698
Train Epoch: 11 [58000/60000 (97%)]	Loss: 0.000624

Test set: Average loss: 0.0085, Accuracy: 9939/10000 (99%)

Train Epoch: 12 [0/60000 (0%)]	Loss: 0.000600
Train Epoch: 12 [2000/60000 (3%)]	Loss: 0.000699
Train Epoch: 12 [4000/60000 (7%)]	Loss: 0.000934
Train Epoch: 12 [6000/60000 (10%)]	Loss: 0.000585
Train Epoch: 12 [8000/60000 (13%)]	Loss: 0.000593
Train Epoch: 12 [10000/60000 (17%)]	Loss: 0.001310
Train Epoch: 12 [12000/60000 (20

KeyboardInterrupt: 

In [ ]:
# initialize network classes
#capsnet = CapsNet(3, 10)
#reconstructionnet = ReconstructionNet(16, 10)
#model = CapsNetWithReconstruction(capsnet, reconstructionnet)